Hemer Jan, Pavlovský Jakub
# Prove of concept
---
This code serves as a proof of feasibility of the project. We have succesfully managed to scrape data from the *www.fortunaliga.cz* webiste for 2022/23 football season. this is not the whole dataset as multiple other seasons will be also scraped and merged to the dataset, using a similar method.

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

#If you don't have these packages installed, please install it, otherwise it won't work

/Users/jenda/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
list_number1 = ['150', '0', '50', '50', '150', '200', '250']
list_number2 = ['0', '50', '100', '150', '200', '250', '300']
col_names = ['Zápas', 'G', 'G1p', 'G2p', 'ÚT', 'GR', 'S', 'SnB', 'P', 'P%', 'ŽK', 'ČK', 'F']
merged_df = pd.DataFrame(columns = col_names)

for yea in range(2014, 2025):
    for ln1, ln2 in zip(list_number1, list_number2):
        response = requests.get(f"https://www.fortunaliga.cz/statistiky?unit=4&status=&parameter=1&season={yea}&club=0&game_limit=&nationality=&age=&position=&list_number={ln1}&order=2&order_dir=1&list_number={ln2}")
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            table = soup.find('table', class_ = 'table')
        
            titles = soup.find_all('abbr')
            titles = [element.text for element in titles]

            content = table.find_all('tr')
            #print(type(content))
            content = [table.find_all('td') for element in content]
            content = [str(element.text) for result_set in content for element in result_set]
            #print(type(content))

            num_elements_per_row = 14
            num_rows = len(content) // num_elements_per_row
            rows = [content[i:i + num_elements_per_row] for i in range(0, len(content), num_elements_per_row)]
            df = pd.DataFrame(rows, columns = [f'Column_{i+1}' for i in range(num_elements_per_row)])
            df = df.set_index('Column_1')
            df.columns = titles
            df = df.reset_index()
            df = df.drop_duplicates(subset = ['Column_1'])

            merged_df = pd.concat([merged_df, df], axis = 0, ignore_index = True)

In [5]:
merged_df.head()

,Zápas,G,G1p,G2p,ÚT,GR,S,SnB,P,P%,ŽK,ČK,F,Column_1
0,"FKJ-BRN 0:0, 13/14",0,0,0,163,0,23,6,862,"75,2%",5,1,28,1.
1,"ZNO-TEP 0:0, 13/14",0,0,0,168,0,27,5,957,"76,1%",5,0,31,2.
2,"PLZ-ACS 0:0, 13/14",0,0,0,183,0,21,4,891,"78,6%",4,0,30,3.
3,"PLZ-DUK 0:0, 13/14",0,0,0,160,0,28,8,962,"78,2%",2,0,23,4.
4,"BOH-JIH 0:0, 13/14",0,0,0,185,0,25,8,901,"80,9%",4,0,26,5.


In [6]:
extracted_columns = merged_df['Zápas'].str.extract(r'([A-Z]+)-([A-Z]+) (\d+):(\d+), (\d{2}/\d{2})')
extracted_columns.columns = ['Home', 'Away', 'Home_Goals', 'Away_Goals', 'Season']
extracted_columns['Match'] = extracted_columns['Home'] + '-' + extracted_columns['Away']
extracted_columns

,Home,Away,Home_Goals,Away_Goals,Season,Match
0,FKJ,BRN,0,0,13/14,FKJ-BRN
1,ZNO,TEP,0,0,13/14,ZNO-TEP
2,PLZ,ACS,0,0,13/14,PLZ-ACS
3,PLZ,DUK,0,0,13/14,PLZ-DUK
4,BOH,JIH,0,0,13/14,BOH-JIH
...,...,...,...,...,...,...
2749,FCS,BOH,5,2,23/24,FCS-BOH
2750,FCZ,PLZ,1,7,23/24,FCZ-PLZ
2751,PLZ,PCE,6,2,23/24,PLZ-PCE
2752,LIB,FCZ,5,3,23/24,LIB-FCZ


In [9]:
final_df = pd.concat([extracted_columns, merged_df.drop('Zápas', axis=1)], axis=1)
final_df

,Home,Away,Home_Goals,Away_Goals,Season,Match,G,G1p,G2p,ÚT,GR,S,SnB,P,P%,ŽK,ČK,F,Column_1
0,FKJ,BRN,0,0,13/14,FKJ-BRN,0,0,0,163,0,23,6,862,"75,2%",5,1,28,1.
1,ZNO,TEP,0,0,13/14,ZNO-TEP,0,0,0,168,0,27,5,957,"76,1%",5,0,31,2.
2,PLZ,ACS,0,0,13/14,PLZ-ACS,0,0,0,183,0,21,4,891,"78,6%",4,0,30,3.
3,PLZ,DUK,0,0,13/14,PLZ-DUK,0,0,0,160,0,28,8,962,"78,2%",2,0,23,4.
4,BOH,JIH,0,0,13/14,BOH-JIH,0,0,0,185,0,25,8,901,"80,9%",4,0,26,5.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2749,FCS,BOH,5,2,23/24,FCS-BOH,7,4,3,87,3,31,11,684,"73,5%",4,0,29,144.
2750,FCZ,PLZ,1,7,23/24,FCZ-PLZ,8,5,3,87,6,33,15,733,"84,7%",3,1,20,145.
2751,PLZ,PCE,6,2,23/24,PLZ-PCE,8,4,4,78,4,28,11,658,"79,2%",5,0,25,146.
2752,LIB,FCZ,5,3,23/24,LIB-FCZ,8,3,5,74,2,20,12,869,"83,2%",5,0,23,147.
